### Universidad del Valle de Guatemala
#### Brandon Ronaldo Sicay Cumes - 21757
##### Modelo base, métricas personalizadas, función optimización




## **Proyecto Final: Detectar patrones secuenciales de fraude**

Descripción del problema: Optimizar el modelo para detectar fraudes que ocurren en secuencia o con patrones.

## Modelo Base


In [3]:
%pip install lightgbm


Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 1.6 MB 1.2 MB/s eta 0:00:01
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd


# Preparación y Separación de Datos

## 1. Carga de Datos

Importamos el dataset preprocesado con las nuevas variables creadas.





In [4]:
# Cargar el dataset final con nuevas features
df = pd.read_csv('./docs/Caracteristicas_final.csv')

/var/folders/cy/x57dh4ds2g123w7nj93wcf_c0000gn/T/ipykernel_61930/3659505003.py:2: DtypeWarning: Columns (39,42,48,64) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./docs/Caracteristicas_final.csv')



## 2. Separación de la Data

Dividimos los datos de entrenamiento y prueba siguiendo una separación temporal, utilizando como testing el último trimestre (octubre a diciembre) de 2020.




In [5]:
# Creamos un filtro para test: último trimestre de 2020
test_mask = (df['trans_month'].isin([10, 11, 12])) & (df['year'] == 2020)

# División en train y test
df_train = df[~test_mask]
df_test = df[test_mask]



## 3. Definición de Variables Dependientes e Independientes
Separamos las variables predictoras (features) y la variable objetivo (is_sequential_fraud_pattern).








In [6]:
# Definir X e y
X_train = df_train.drop(columns=['is_sequential_fraud_pattern'])
y_train = df_train['is_sequential_fraud_pattern']

X_test = df_test.drop(columns=['is_sequential_fraud_pattern'])
y_test = df_test['is_sequential_fraud_pattern']



## 4. Identificación de Variables Categóricas y Numéricas
Identificamos las variables categóricas y numéricas del dataset.










In [7]:
# Columnas categóricas y numéricas
categorical_cols = X_train.select_dtypes(include=['object', 'category']).columns
numerical_cols = X_train.select_dtypes(include='number').columns

print("Variables categóricas:", list(categorical_cols))
print("Variables numéricas:", list(numerical_cols))


Variables categóricas: ['timestamp', 'merchant', 'category', 'first', 'last', 'gender', 'street', 'city', 'state', 'job', 'dob', 'trans_num', 'unix_time', 'secs_since_last_trans', 'fraud_secs_gap_x', 'min_fraud_gap_secs_x', 'mean_fraud_gap_secs_x', 'fraud_secs_gap_y', 'min_fraud_gap_secs_y', 'mean_fraud_gap_secs_y', 'last_fraud_time', 'days_since_last_fraud', 'fraud_secs_gap', 'min_fraud_gap_secs', 'mean_fraud_gap_secs', 'date', 'time_since_last_event', 'fraud_to_fraud_time', 'fraud_legit_gap']
Variables numéricas: ['cc_num', 'amt', 'zip', 'lat', 'long', 'city_pop', 'merch_lat', 'merch_long', 'is_fraud', 'amt_month', 'amt_year', 'amt_month_shopping_net_spend', 'count_month_shopping_net', 'dist_between_client_and_merch', 'trans_month', 'trans_day', 'hour', 'year', 'times_shopped_at_merchant', 'times_shopped_at_merchant_year', 'times_shopped_at_merchant_month', 'times_shopped_at_merchant_day', 'trans_rolling_1h', 'rolling_amt_mean_1h', 'trans_dist_km', 'daily_amt_std', 'mean_amt', 'std_a


## 5. Procesamiento de Variables Categóricas
Codificamos las variables categóricas usando Label Encoding para que LightGBM pueda procesarlas adecuadamente. Además, manejamos valores < unknown > en el conjunto de prueba para valores no vistos durante el entrenamiento.









### Evaluación de modelo

In [ ]:
from sklearn.preprocessing import LabelEncoder
import bisect
import numpy as np

# Copiar datasets
X_train_processed = X_train.copy()
X_test_processed = X_test.copy()

# Procesar variables categóricas
for col in categorical_cols:
    le = LabelEncoder()
    X_train_processed[col] = le.fit_transform(X_train[col].astype(str))
    
    # Mapeo especial para valores no vistos
    X_test_processed[col] = X_test[col].astype(str).map(lambda s: '<unknown>' if s not in le.classes_ else s)

    if '<unknown>' not in le.classes_:
        if X_test_processed[col].eq('<unknown>').any():
            le_classes = le.clases_.tolist()
            bisect.insort_left(le_classes, '<unknown>')
            le.classes_ = np.array(le_classes)
    
    X_test_processed[col] = le.transform(X_test_processed[col])

### Entrenamiento del Modelo Base


In [ ]:
import lightgbm as lgb

params = {
    'objective': 'binary',
    'metric': 'auc',
    'boosting_type': 'gbdt',
    'learning_rate': 0.05,
    'num_leaves': 31,
    'max_depth': -1,
    'is_unbalance': True,
    'n_jobs': 4,
    'seed': 42
}

train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

# Entrenar modelo inicial
model_base = lgb.train(
    params,
    train_data,
    num_boost_round=1000,
    valid_sets=[test_data],
    early_stopping_rounds=50,
    verbose_eval=100
)
